In [ ]:
# ===============================
# Phase 4: Real Estate + LLaMA Integration
# ===============================

# Install required packages (run once)
%pip install pandas numpy scikit-learn requests ipykernel

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans  # Fixed typo (was KMeansS)
from sklearn.metrics.pairwise import euclidean_distances
import requests
import os
from IPython.display import Markdown, display

# ----------------------------
# 1. Load Dataset (with error handling)
# ----------------------------
try:
    file_path = "Dataset/cleaned_dataset.csv"
    df = pd.read_csv(file_path)
    
    # Preserve original values
    df["Original_Area"] = df["Area"]
    df["Original_Price"] = df["Price"]
except Exception as e:
    raise FileNotFoundError(f"Failed to load dataset: {str(e)}")

# ----------------------------
# 2. Preprocessing
# ----------------------------
features = ["Area", "Price"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# ----------------------------
# 3. Apply KMeans Clustering
# ----------------------------
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# ----------------------------
# 4. Make Recommendation (based on cluster)
# ----------------------------
def recommend_properties_kmeans(price_input, area_input, top_n=5):
    try:
        input_df = pd.DataFrame([[area_input, price_input]], columns=["Area", "Price"])
        input_scaled = scaler.transform(input_df)
        input_cluster = kmeans.predict(input_scaled)[0]

        cluster_data = df[df["Cluster"] == input_cluster].copy()
        cluster_scaled = scaler.transform(cluster_data[["Area", "Price"]])
        distances = euclidean_distances(input_scaled, cluster_scaled)[0]

        cluster_data["Distance"] = distances
        recommended = cluster_data.sort_values("Distance").head(top_n)

        recommended = recommended.rename(columns={
            "Property Type": "نوع العقار",
            "Location": "الموقع",
            "District": "الحي",
            "Bedrooms": "الغرف",
            "Bathrooms": "دورات المياة",
            "Original_Area": "المساحة",
            "Original_Price": "السعر",
            "Agency_Name": "الوكالة"
        })

        headers = ["نوع العقار", "الموقع", "الحي", "الغرف", "دورات المياة", "المساحة", "السعر", "الوكالة"]
        return recommended[headers]
    except Exception as e:
        raise ValueError(f"Recommendation failed: {str(e)}")

# ----------------------------
# 5. Prompt Templates
# ----------------------------
def template_informative(properties):
    explanation = "Here are the top recommended properties based on your preferences:\n\n"
    for i, prop in enumerate(properties):
        explanation += (
            f"{i+1}. Located in {prop['الموقع']} - حي {prop['الحي']}, this {prop['نوع العقار']} includes "
            f"{prop['الغرف']} rooms and {prop['دورات المياة']} bathrooms, covering {prop['المساحة']} sqm, "
            f"priced at {prop['السعر']} SAR. Listed by {prop['الوكالة']}.\n"
        )
    explanation += "\nThese suggestions are selected based on similarity in area and budget using KMeans clustering."
    return explanation

def template_friendly(properties):
    response = "Hey! Based on what you're looking for, here are some cool real estate picks:\n\n"
    for i, prop in enumerate(properties):
        response += (
            f"{i+1}. A lovely {prop['نوع العقار']} in {prop['الموقع']} (حي {prop['الحي']}) with {prop['الغرف']} rooms "
            f"and {prop['دورات المياة']} bathrooms. It's {prop['المساحة']} sqm and goes for about {prop['السعر']} SAR. "
            f"Offered by {prop['الوكالة']}.\n"
        )
    response += "\nLet me know if you'd like to check one out or get more info!"
    return response

# ----------------------------
# 6. Call LLaMA via Together API (Secure Version)
# ----------------------------
def call_llama(prompt, model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.7):
    try:
        TOGETHER_API_KEY = os.getenv("6c17b9ec54233e8f0b39af461845214728ac7a80f59e94c2dfb122970f2464ed")
        if not TOGETHER_API_KEY:
            raise ValueError("API key not found. Set TOGETHER_API_KEY environment variable.")
            
        headers = {
            "Authorization": f"Bearer {TOGETHER_API_KEY}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": model,
            "prompt": prompt,
            "max_tokens": 300,
            "temperature": temperature,
            "top_p": 0.9,
            "repetition_penalty": 1.1,
        }

        response = requests.post(
            "https://api.together.xyz/v1/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()
        return response.json()["choices"][0]["text"]
    except Exception as e:
        raise Exception(f"API call failed: {str(e)}")

# ----------------------------
# 7. Run Example (with validation)
# ----------------------------
try:
    # Example inputs
    price_input = 980000
    area_input = 300
    
    # Validate inputs
    if not isinstance(price_input, (int, float)) or not isinstance(area_input, (int, float)):
        raise ValueError("Price and area must be numbers")
    
    # Get recommendations
    recommended = recommend_properties_kmeans(price_input, area_input)
    property_dicts = recommended.to_dict(orient="records")

    # Generate prompts
    prompt1 = template_informative(property_dicts)
    prompt2 = template_friendly(property_dicts)

    # Call LLM (with error handling)
    response1 = call_llama(prompt1) if len(property_dicts) > 0 else "No properties found"
    response2 = call_llama(prompt2) if len(property_dicts) > 0 else "No properties found"

    # Display results
    display(Markdown("### Template 1: Informative Style"))
    display(Markdown(response1.strip()))

    display(Markdown("### Template 2: Conversational Style"))
    display(Markdown(response2.strip()))

except Exception as e:
    print(f"Error in main execution: {str(e)}")